# Path Planning Strategies

In [1]:
# Import packages.
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

import import_ipynb
import utils

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from node_linker.ipynb


## Analytical Geometric Algorithm Implementation

`def aga(agent)`

Simple algorithm implementation to get the closest point and edges for each
robot in its cell.

### Parameters:
* **agent : class machines.Robot**
    * Current agent.

In [ ]:
def aga(agent):
    edge_points = utils.perpendicular_dist(agent.pos_f, agent.bvc)
    target_point = np.array(agent.pos_f)
    vertices = np.array(agent.bvc + edge_points)
    # Initialize
    agent.closer_point = np.linalg.norm(vertices[0] - target_point)
    pointer = 0
    # Review the rest of vertices
    for j in range(1, len(vertices)):
        # Measure the distance of the candidate vertex to the target
        candidate = np.linalg.norm(vertices[j] - target_point)
        # Take the closest point
        if candidate < agent.closer_point:
            agent.closer_point = candidate
            pointer = j
    # Update the closest point 
    agent.closer_point = vertices[pointer]
    get_intersection = False
    k = 0
    while not get_intersection and k < len(agent.bvc)-1:
        agent.closer_point = utils.sharpen(agent.pos_f, agent.bvc[k], agent.pos_i,
                                           agent.bvc[k+1])
        is_in_edge = utils.between(agent.bvc[k+1], agent.bvc[k],
                                   agent.closer_point)
        is_in_path = utils.between(agent.pos_f, agent.pos_i,
                                   agent.closer_point)
        if is_in_edge and is_in_path:
            get_intersection = True
            agent.closer_edge = [agent.bvc[k+1], agent.bvc[k]]
        else:
            k += 1
    if not get_intersection:
        agent.closer_edge = [[0,0],[0,0]]

## Controller Function

`def controller(opt, radius, robots, step)`

Function to generate the closest point and edges according to the algorithm
selected

### Parameters:
* **opt&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: dictionary**
    * Dictionary with general options
* **radius :&nbsp;float**
    * Safety radius.
* **robots&nbsp;:&nbsp;list**
    * List of instances of Robots
* **step&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;int**
    * Global step of the execution

In [ ]:
def controller(opt, radius, robots, step):
    for agent in robots:
        if agent.inside:
            agent.closer_point = agent.pos_f
        else:
            if opt["optimize"]:
            # Calculate the closest point using the QP-based RHC algorithm
                rhc(agent, opt, radius, step)
            # Calculate the closest point using the AGA
            else:
                aga(agent)
            if opt["verbose"]:
                print('Closest point:', agent.closer_point)

## QP-based RHC Solver Implementation
`def optimizer(agent, opt, plot_flag, radius, step)`

Receding Horizon Path Planning Solver

### Parameters:
* **agent&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: class machines.Robot**
    * Current agent.
* **opt&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: dictionary**
    * Dictionary with general options
* **plot_flag : bool**
    * True to plot the construction of the optimal value
* **radius&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: float**
    * Safety radius.
* **step&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: int**
    * Global step of the execution

In [20]:
def optimizer(agent, opt, plot_flag, radius, step):
    np.random.seed(1)
    # Dimensions
    T = 20 # Steps
    m = 2 # u dimension
    n = 2 # x dimension
    # Mechanics
    alpha = 0.2
    A = np.eye(n) + alpha*np.random.randn(n,n)
    B = np.random.randn(n,m)
    # Parameters
    R_sqrt = cp.Parameter((m,m), PSD = True)
    Q_sqrt = cp.Parameter((n,n), PSD = True)
    Qf_sqrt = cp.Parameter((n,n), PSD = True)
    R_sqrt.value = np.identity(m)
    Q_sqrt.value = np.identity(n)
    Qf_sqrt.value = np.identity(n)
    # Variables
    p = cp.Variable((n,T+1))
    u = cp.Variable((m,T))
    u_max_x = 1
    u_max_y = 1
    # Artificial operations for DPP
    R_sim = np.random.randn(m,m)
    Q_sim = np.random.randn(n,n)
    Qf_sim = np.random.randn(n,n)
    R_inst = R_sim.T @ R_sim
    Q_inst = Q_sim.T @ Q_sim
    Qf_inst = Qf_sim.T @ Qf_sim
    # Cost and Constraints
    cost = 0
    constr = []
    constr += [p[:,0] == agent.pos_i]
    for t in range(T):
        cost += cp.sum_squares(Q_sqrt @ p[:,t]-agent.pos_f) + cp.sum_squares(R_sqrt @
                                                                     u[:,t])
        constr += [p[:,t+1] == A@p[:,t] + B@u[:,t],
                   cp.abs(u[0,t]) <= u_max_x,
                   cp.abs(u[1,t]) <= u_max_y]
        for robot in agent.neighbors_pos:
            p_ij = (robot - agent.pos_i)
            temp = (cp.transpose(p[:,t] - (agent.pos_i + robot)/2) @ p_ij
                    + (radius * np.linalg.norm(p_ij)) <= 0)
            #print(temp)
            constr += [temp]
    # Last case
    cost += cp.sum_squares(Qf_sqrt @ p[:,T]-agent.pos_f)
    for robot in agent.neighbors_pos:
        p_ij = (robot - agent.pos_i)
        constr += [(cp.transpose(p[:,T] - (agent.pos_i + robot)/2) @ p_ij
                    + (radius * np.linalg.norm(p_ij)) <= 0)]
    #Assing values
    R_sqrt.value = scipy.linalg.sqrtm(R_inst)
    Q_sqrt.value = scipy.linalg.sqrtm(Q_inst)
    Qf_sqrt.value = scipy.linalg.sqrtm(Q_inst) #For accuracy
    # Objective
    prob = cp.Problem(cp.Minimize(cost), constr)
    # Solve with no Disciplined Geometric Programming
    #prob.solve(solver=cp.ECOS, gp=False, verbose=True)
    prob.solve(solver=cp.ECOS, gp=False)
    # Optimal solution
    optimal_value = prob.value
    #Graphics
    if plot_flag:
        plot_rhc(u, p, opt, step)
    # Check status
    #print("We get the ", prob.status, " value")
    # Return
    if opt["verbose"]:
        print(prob.status)
    agent.closer_point = p.value[:,T]

## Receding Horizons Path Planning Function

`def rhc(agent, opt, radius, robots, step)`

Receding Horizon Path Planning basics with the solver embedded

#### Parameters:
* **agent&nbsp;&nbsp;: class machines.Robot**
    * Current agent.
* **opt&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: dictionary**
    * Dictionary with general options
* **radius : float**
    * Safety radius.
* **step&nbsp;&nbsp;&nbsp;&nbsp;: int**
    * Global step of the execution

In [ ]:
def rhc(agent, opt, radius, step):
    if opt["verbose"]:
        print("Robots in front:")
        for k in agent.neighbors_pos:
            print(k)
    # Calculate the closest point using the CVXPY Solver
    optimizer(agent, opt, False, radius, step)
    get_intersection = False
    i = 0
    while not get_intersection and i < len(agent.bvc)-1:
        if utils.between(agent.bvc[i+1], agent.bvc[i], agent.closer_point):
            get_intersection = True
            agent.closer_edge = [agent.bvc[i+1], agent.bvc[i]]
        else:
            i += 1
    if not get_intersection:
        agent.closer_edge = [[0,0], [0,0]]